# Initialization (Enviroments and Hyperparameters)

In [ ]:
import os
import numpy as np
import random
import math
import itertools
import copy
import nltk
import pickle
nltk.download('punkt')
from transformers import BertConfig,BertForSequenceClassification, BertModel, BertTokenizer
import torchvision.transforms as transforms
from torchvision.utils import save_image
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
from skimage import io
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch
cuda = True if torch.cuda.is_available() else False

# Set random seed for reproducibility
manualSeed = 999
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
class Object(object):
    pass

opt = Object()
opt.n_epochs = 30000 # number of epochs of training
opt.batch_size = 32 # size of the batches
opt.lr = 0.0008 # adam: learning rate
opt.b1 = 0.7 # adam: decay of first order momentum of gradient
opt.b2 = 0.999 # adam: decay of first order momentum of gradient
opt.n_cpu = 8 # number of cpu threads to use during batch generation
opt.latent_dim = 100 # dimensionality of the latent space
opt.img_size = 64 # size of each image dimension
opt.channels = 3 # number of image channels
opt.sample_interval = 1000 # interval between image sampling
img_shape = (opt.channels, opt.img_size, opt.img_size)
opt.embedding_dim = 300 # embedding's dimension, BERT:768,  google-word2vec:300
opt.reduced_embedding = 100
opt_g = copy.copy(opt)
opt_d = copy.copy(opt)
opt_d.lr = 0.0001
opt_d.b1 = 0.4
opt_d.b2 = 0.99

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')
NLP_MODEL = BertModel.from_pretrained('bert-base-uncased',
                                      output_attentions = False, 
                                      output_hidden_states = True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ROOT_DIR = "/content/drive/My Drive/emoji_data/"


In [ ]:
def load_variable(file_name):
  with open(ROOT_DIR + "variables/" + file_name, "rb") as fb:
    data = pickle.load(fb)
    fb.close()
    return data
GOOGLE_EMBEDDING = load_variable("google_embedding.pkl")
BERT_EMBEDDING = load_variable("bert_embedding.pkl")
EMBEDDINGS_LIST = load_variable("embeddings_list.pkl")


# Example of for Loading Data

In [ ]:
import json
emoji_json = open(ROOT_DIR + "data.json")
emoji_data = json.load(emoji_json)

emoji_data.items()

In [ ]:
#print(emoji_data)
emoji_test = emoji_data['img-facebook-64//1f600.png']
print(emoji_test)

In [ ]:
full_names = []
for key,value in emoji_data.items():
  emoji = emoji_data[key]
  full_names.append(emoji[0]['full_name'])

print(full_names)

In [ ]:
emoji_info = emoji_data['img-facebook-64//1f600.png']
full_name = emoji_info[0]['full_name']
print(full_name)

In [ ]:
tokens = TOKENIZER.tokenize(full_name)
indexed_tokens = TOKENIZER.convert_tokens_to_ids(tokens)
embeddings = TOKENIZER.encode(tokens,
                              add_special_tokens = False,
                              return_attention_mask = True,   # Construct attn. masks. 
                              return_tensors = 'pt'   # Return pytorch tensors.
)
print(embeddings)
print(device)

In [ ]:
hidden_output = NLP_MODEL(torch.tensor([indexed_tokens]))

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
def get_text_embeddings(text, src):
  if src == 'google':
    sentence_embedding = []
    tokens = text.replace("-", " ")
    tokens = tokens.split(' ')
    for token in tokens:
      if token.lower() not in WORD_VEC.vocab:
        #print(token, " is not in vocab")
        continue
      token_embedding = WORD_VEC[token.lower()]
      sentence_embedding.append(token_embedding)
    #print(np.array(sentence_embedding).shape)
    avg_embeddings = np.mean(np.array(sentence_embedding), 0)
  elif  src == 'bert':
    tokens = TOKENIZER.tokenize(text)
    indexed_tokens = TOKENIZER.convert_tokens_to_ids(tokens)
    if device.type == 'cuda':
      hidden_output = NLP_MODEL(torch.tensor([indexed_tokens]))
    else:  
      hidden_output = NLP_MODEL(torch.tensor([indexed_tokens]))
    embeddings = hidden_output[0][0]
    avg_embeddings = torch.mean(embeddings, dim = 0)
    return avg_embeddings
  elif src == 'google_list':
    avg_embeddings = GOOGLE_EMBEDDING[text]
  elif src == 'bert_list':
    avg_embeddings = BERT_EMBEDDING[text]
  return FloatTensor(avg_embeddings)


In [ ]:
#test_embedding = get_text_embeddings("you do want", "google_list")
#print(test_embedding.shape)

In [ ]:
def get_all_embeddings(emoji_data, root_dir, src):
  full_names = []
  image_paths = []
  embeddings_list = []
  for key,value in emoji_data.items():
      emoji = emoji_data[key]
      full_name = emoji[0]["full_name"]
      embedding = get_text_embeddings(full_name, src)
      embeddings_list.append(embedding.cpu().data.numpy())
      full_names.append(full_name)# loaded json data as input
      img_path = emoji[0]['path']
      image_paths.append( root_dir+ '//'+ img_path)
  return full_names, image_paths, np.array(embeddings_list)

full_names , image_paths, embeddings_list = get_all_embeddings(emoji_data, ROOT_DIR,'google_list')
#embeddings_list = np.transpose(np.array(embeddings_list))

In [ ]:
print(embeddings_list.shape)

In [ ]:
def generate_embeddings_list(emoji_datesets, src):
  saved_embeddings_list = {}

  for key, value in emoji_data.items():
      emoji = emoji_data[key]
      full_name = emoji[0]["full_name"]
      if full_name not in saved_embeddings_list.keys():
         embedding = get_text_embeddings(full_name, src)
         saved_embeddings_list[full_name] = embedding.cpu().data.numpy()
  return saved_embeddings_list

def load_embeddings_list(file_name):
   out = []
   return out

# Customize Dataset and Dataloader

In [ ]:
# Customize own dataset
class EmojiDataset(Dataset):
    """smiley emoji dataset."""

    def __init__(self, emoji_data, root_dir=ROOT_DIR, source="google_list", transform=None):
        """
        Args:
            json_file (string): Path to the json file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.src = source
        self.full_names , self.image_paths, self.embedding_list = get_all_embeddings(emoji_data,root_dir = ROOT_DIR, src= source)
        self.emoji_data = emoji_data
        self.root_dir = root_dir
        self.transform = transform
        

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
      
        image = cv2.imread(self.image_paths[idx]) / 255 # IO OUTPUT IMAGE WITH 4 CHANNELS

        name = self.full_names[idx]

        embeddings = get_text_embeddings(name, self.src)

        sample = {'image': image, 'embeddings': embeddings}

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, embeddings = sample['image'], sample['embeddings']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': Tensor(image),
                'embeddings': FloatTensor(embeddings)}
                
class Rescale(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['embeddings']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively

        return {'image': img, 'embeddings': embeddings}

emoji_datesets = EmojiDataset(emoji_data, ROOT_DIR, source="google_list", transform=transforms.Compose([
                                               ToTensor()]))

In [ ]:
test = {}
'face' in test.keys()

In [ ]:
for i in range(len(emoji_datesets)):
    sample = emoji_datesets[i]
    #print(sample)
    print(i, sample['image'].size(), sample['embeddings'].size())

    if i == 3:
        break

In [ ]:
dataloader = DataLoader(emoji_datesets, batch_size= opt.batch_size ,
                        shuffle=True)

In [ ]:
batch_test = []
embedding_test = []
for i, sample_batched in enumerate(dataloader):
  images_batch, embeddings_batch = sample_batched['image'], sample_batched['embeddings']
  print(images_batch.size())
  print(embeddings_batch.size())
  if i == 1:
    batch_test = images_batch
    embedding_test = embeddings_batch
    break
    

In [ ]:
t = embedding_test.unsqueeze(1)
t = t.unsqueeze(2)
t.repeat(1,4,4,1).size()

# Discriminator

In [ ]:
from torch.nn import functional as func
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def conv_block(in_filters, out_filters, bn=True):
            #block = [nn.Conv2d(in_filters, out_filters,3,2,1), nn.LeakyReLU(0.2, inplace=True)]
            if bn:
                block = [nn.Conv2d(in_filters, out_filters,3, 2, 1, padding_mode="replicate"),
                         nn.BatchNorm2d(out_filters, 0.9), nn.LeakyReLU(0.1, inplace=True)]
            else:
                block = [nn.Conv2d(in_filters, out_filters,3, 2, 1),nn.BatchNorm2d(out_filters, 0.9), nn.LeakyReLU(0.2, inplace=True)]
            return block

        def linear_block(in_filters, out_filters, activation = None):
            block = [nn.Linear(in_filters, out_filters)]
            if activation == 'relu':
                block.append(nn.LeakyReLU(0.1, inplace = True))
            elif activation  == 'sigmoid':
                block.append(nn.Sigmoid())
            return block
        
        self.pre_fc = nn.Sequential(nn.Linear(opt.embedding_dim, opt.reduced_embedding),
                      nn.BatchNorm1d(opt.reduced_embedding, 0.9), nn.LeakyReLU(0.1, inplace=True))               

        self.conv_model = nn.Sequential(
            *conv_block(opt.channels, 64),
            *conv_block(64, 128),
            *conv_block(128, 256),
            *conv_block(256, 512),
        )

        self.fc1 = nn.Sequential(nn.Conv2d(512+ opt.reduced_embedding ,512, 1, 1, padding_mode="replicate"), nn.BatchNorm2d(512, 0.9), nn.LeakyReLU(0.1, inplace=True))

        self.fc3 = nn.Sequential(nn.Linear(4 * 4 * 512, 1), nn.Sigmoid()) 
        # The height and width of downsampled image
        #ds_size = opt.img_size // 2 ** 4
        #self.adv_layer = nn.Linear(128 * ds_size ** 2, 1)

    def forward(self, image, embeddings): 
        #print('loaded image size: ',image.size())  
        image = self.conv_model(image)
        # dimension adjustion 
        embeddings = embeddings.unsqueeze(2)
        embeddings = embeddings.unsqueeze(3)
        embeddings = embeddings.repeat(1,1,4,4)

        cat_input = torch.cat([image, embeddings], 1)
)
        out = self.fc1(cat_input)

        out = out.view(out.shape[0], -1)

        out = self.fc3(out)
  
        return out

# Generator

In [ ]:
class CGenerator(nn.Module):
    def __init__(self):
        super(CGenerator, self).__init__()
        self.dim = opt.latent_dim

        self.linear_block = nn.Sequential(nn.Linear(opt.embedding_dim, opt.reduced_embedding), nn.ReLU())
        
        self.l1, self.conv_blocks = self.conv_blocks2()
          
    def forward(self, image, embeddings):
        embeddings = self.linear_block(embeddings)
        cat_input = torch.cat([image, embeddings], 1)

        out = self.l1(cat_input)

        out = out.view(out.shape[0], self.layer1_depth, self.init_size, self.init_size)
        img = self.conv_blocks(out)

        return img
   
    def conv_blocks2(self):
        # ----------------------
        # adjustable parameters
        self.scale = 0.5
        self.init_size = opt.img_size // 4
        self.layer1_depth = int(1024 * self.scale)
        self.layer2_depth = int(512 * self.scale)
        self.layer3_depth = int(256 * self.scale)
        self.layer4_depth = int(128 * self.scale)
        self.layer5_depth = int(64 * self.scale)
        # ----------------------

        l1 = nn.Sequential(nn.Linear(opt.latent_dim + opt.reduced_embedding, self.layer1_depth * self.init_size ** 2), 
                           nn.BatchNorm1d(self.layer1_depth * self.init_size ** 2,0.9), nn.ReLU())
        # output dim: (self.layer1_depth, self.init_size, self.init_size)
        
        conv_block = nn.Sequential(
            # Layer 1
            #nn.LeakyReLU(0.1, inplace=True),
            nn.ConvTranspose2d(self.layer1_depth, self.layer2_depth, 3, stride=2, padding=1, padding_mode="zeros"),
            nn.BatchNorm2d(self.layer2_depth, 0.9),
            nn.ReLU(),
            # Layer 2        
            nn.ConvTranspose2d(self.layer2_depth, self.layer3_depth, 3, stride=2, padding=1, padding_mode="zeros"),
            nn.BatchNorm2d(self.layer3_depth, 0.9),
            nn.ReLU(),
            # output dim: (self.layer3_depth, hh, ww)
            # where hh=ww=(h-1)*stride-2*padding+(kernel_size-1)+1

            # Layer 3        
            nn.Conv2d(self.layer3_depth, self.layer4_depth, 3, stride= 2, padding=1,padding_mode="zeros"),
            nn.BatchNorm2d(self.layer4_depth, 0.9),
            nn.ReLU(),
            # Layer 4            
            nn.ConvTranspose2d(self.layer4_depth, opt.channels, 3, stride= 2, padding= 1, padding_mode="zeros"),
            nn.Sigmoid()
        )
        
        return l1, conv_block

# Training


In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.05)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
# Loss function
#adversarial_loss = torch.nn.MultiLabelMarginLoss()
# dversarial_loss = torch.nn.MSELoss()
adversarial_loss = torch.nn.BCEWithLogitsLoss()
input = torch.FloatTensor(opt.batch_size, opt.channels*64*64)
noise = torch.FloatTensor(opt.batch_size, opt.latent_dim)

# Initialize generator and discriminator
generator = CGenerator()
print(generator)
discriminator = Discriminator()
print(discriminator)
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt_g.lr, betas=(opt_g.b1, opt_g.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt_d.lr, betas=(opt_d.b1, opt_d.b2))

In [ ]:
import matplotlib.pylab as plt
def plot_loss(loss_G, loss_D):
    plt.plot(loss_G, 'b',label='Generator Loss ')
    plt.plot(loss_D,'r',label='Discriminator Loss')
    plt.xlabel('iteration')
    plt.title("loss")
    plt.legend()
    plt.show()

In [ ]:
g_loss_list, d_loss_list = [], []
#embeddings_list = load_embedding_list()
m, n = embeddings_list.shape
#torch.autograd.set_detect_anomaly(True)
for epoch in range(opt.n_epochs):
    generator.train()
    discriminator.train()
    gloss = 0
    dloss = 0
    for i, sample_batched in enumerate(dataloader):
        # Adversarial ground truths
        image_batched, embeddings_batched = sample_batched['image'], sample_batched['embeddings']  

        valid = Variable(Tensor(image_batched.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(image_batched.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(Tensor(image_batched))
        real_embeddings = Variable(FloatTensor(embeddings_batched))

        # -----------------
        #  Train Generatorr
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (image_batched.shape[0], opt.latent_dim))))
    
        choose_idx = np.random.choice(m, image_batched.shape[0])
        choose_embeddings = embeddings_list[choose_idx, :]

        fake_embeddings = Variable(FloatTensor(choose_embeddings))
        #print(z.size(),fake_embeddings.size())
        # Generate a batch of images
        gen_imgs = generator(z, real_embeddings)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, real_embeddings)
        #g_loss = adversarial_losss(discriminator(gen_img, gen_embeddings), valid)
        g_loss = adversarial_loss(validity, valid)
        gloss = g_loss.item() + gloss
        g_loss.backward()        
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, real_embeddings)
        d_real_loss = adversarial_loss(validity_real, valid)

        #validity_real_fake = discriminator(real_imgs, fake_embeddings)
        #d_real_fake_loss = adversarial_loss(validity_real_fake,fake)

        validity_fake = discriminator(gen_imgs.detach(), real_embeddings)
        d_fake_loss = adversarial_loss(validity_fake, fake)
        # Loss for fake images


        # Total discriminator loss
        #d_loss = d_fake_loss + (d_real_loss + d_real_fake_loss) / 2
        d_loss = 0.5*(d_fake_loss + d_real_loss)
        # Measure discriminator's ability to classify real from generated samples
        # real_loss = adversarial_loss(discriminator(real_imgs, real_embeddings), valid)
        # fake_loss = adversarial_loss(discriminator(gen_imgs.detach(), gen_embeddings), fake)
        # d_loss = 0.5 * (real_loss + fake_loss)
        #print(d_fake_loss.item(), d_real_fake_loss.item(), d_loss.item())
        dloss = d_loss.item() + dloss
        if d_loss.item()/g_loss.item() > 0.1:
          d_loss.backward()
          optimizer_D.step()
        
        

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i + 1
        # if batches_done % opt.sample_interval == 0:
        #     save_image(gen_imgs.data[:25], "/content/drive/My Drive/results/2/sigmoid/%d.png" % batches_done, nrow=5, normalize=True)
        #     # save_image(real_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        
        # if batches_done % 10000 == 0:
        #     torch.save(generator.state_dict(), "/content/drive/My Drive/results/2/sigmoid/" + "gen_" + str(batches_done))
        #     torch.save(discriminator.state_dict(), "/content/drive/My Drive/results/2/sigmoid/" + "dis_" + str(batches_done))
    g_loss_list.append(gloss)
    d_loss_list.append(dloss)

## Backup Functions

In [ ]:
import gensim
from gensim.models import KeyedVectors

WORD_VEC = KeyedVectors.load_word2vec_format(ROOT_DIR + "google_embedding/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
print(WORD_VEC['happy'])

In [ ]:
google_embeddings_dict = generate_embeddings_list(emoji_data, 'google')
bert_embeddings_dict = generate_embeddings_list(emoji_data, 'bert')

with open(ROOT_DIR + "variables/google_embedding.pkl", 'wb') as fp:
    pickle.dump(google_embeddings_dict, fp)

with open(ROOT_DIR + "variables/bert_embedding.pkl",  'wb') as fp:
    pickle.dump(bert_embeddings_dict, fp)

with open(ROOT_DIR + "variables/embeddings_list.pkl",  'wb') as fp:
    pickle.dump(embeddings_list, fp)